In [2]:
import gzip
import os
import random
from collections import defaultdict

import pandas as pd

In [3]:
def parse(path):
    g = gzip.open(path, "rb")
    for line in g:
        yield eval(line)


def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient="index")

In [4]:
def filter_data(filePath):
    # data = []
    # ratings = pd.read_csv(filePath, delimiter=",", encoding="latin1")
    ratings = getDF(filePath)
    ratings = ratings[["reviewerID", "asin", "overall", "unixReviewTime"]]

    ratings.columns = ["userId", "itemId", "Rating", "timestamp"]
    ratings = ratings.loc[ratings["Rating"] >= 3.0]
    ratings = ratings.drop_duplicates()

    rate_size_dic_i = ratings.groupby("itemId").size()
    choosed_index_del_i = rate_size_dic_i.index[rate_size_dic_i < 5]
    ratings = ratings[~ratings["itemId"].isin(list(choosed_index_del_i))]  # item freq more than 10

    user_unique = list(ratings["userId"].unique())
    movie_unique = list(ratings["itemId"].unique())

    u = len(user_unique)
    i = len(movie_unique)

    rating_num = len(ratings)
    return u, i, rating_num, user_unique, ratings

In [5]:
def filter_user(ratings1, ratings2):
    rate_size_dic_u1 = ratings1.groupby("userId").size()
    rate_size_dic_u2 = ratings2.groupby("userId").size()
    choosed_index_del_u1 = rate_size_dic_u1.index[rate_size_dic_u1 < 60]
    choosed_index_del_u2 = rate_size_dic_u2.index[rate_size_dic_u2 < 60]
    ratings1 = ratings1[~ratings1["userId"].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    ratings2 = ratings2[~ratings2["userId"].isin(list(choosed_index_del_u2) + list(choosed_index_del_u2))]
    return ratings1, ratings2


def filter_item(ratings1, ratings2):
    rate_size_dic_u1 = ratings1.groupby("itemId").size()
    rate_size_dic_u2 = ratings2.groupby("itemId").size()
    choosed_index_del_u1 = rate_size_dic_u1.index[rate_size_dic_u1 < 20]
    choosed_index_del_u2 = rate_size_dic_u2.index[rate_size_dic_u2 < 20]
    ratings1 = ratings1[~ratings1["itemId"].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    ratings2 = ratings2[~ratings2["itemId"].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    return ratings1, ratings2

In [6]:
def sample_data(ratings1, ratings2, ratio1, ratio2):
    overlap_user = list(set(list(ratings1["userId"].unique())) & set(list(ratings2["userId"].unique())))
    non_overlap_ratings1 = ratings1[~ratings1["userId"].isin(overlap_user)]
    non_overlap_ratings2 = ratings2[~ratings2["userId"].isin(overlap_user)]

    ratings1_nolap_user = list(non_overlap_ratings1["userId"].unique())
    ratings2_nolap_user = list(non_overlap_ratings2["userId"].unique())

    samples1 = int(ratio1 * len(ratings1_nolap_user))
    samples2 = int(ratio2 * len(ratings2_nolap_user))

    sampled_user1 = random.sample(ratings1_nolap_user, samples1)
    sampled_user2 = random.sample(ratings2_nolap_user, samples2)

    new_ratings1 = ratings1[ratings1["userId"].isin(overlap_user + sampled_user1)]
    new_ratings2 = ratings2[ratings2["userId"].isin(overlap_user + sampled_user2)]

    return new_ratings1, new_ratings2

In [ ]:
data1_csv = "amazon14/reviews_Movies_and_TV_5.json.gz"
data2_csv = "amazon14/reviews_Digital_Music_5.json.gz"
data3_csv = "amazon14/reviews_Books_5.json.gz"

u, i, rating_num, user_unique1, ratings1 = filter_data(data1_csv)
print(u, i, rating_num)
u, i, rating_num, user_unique2, ratings2 = filter_data(data2_csv)
print(u, i, rating_num)
u, i, rating_num, user_unique3, ratings3 = filter_data(data3_csv)
print(u, i, rating_num)

overlap_user = list(set(user_unique1) & set(user_unique2) & set(user_unique3))

new_ratings1 = ratings1[ratings1["userId"].isin(overlap_user)]
new_ratings2 = ratings2[ratings2["userId"].isin(overlap_user)]
new_ratings3 = ratings3[ratings3["userId"].isin(overlap_user)]
print(len(overlap_user), len(new_ratings1["itemId"].unique()), len(new_ratings1))
print(len(overlap_user), len(new_ratings2["itemId"].unique()), len(new_ratings2))
print(len(overlap_user), len(new_ratings3["itemId"].unique()), len(new_ratings3))

In [ ]:
def save_inter(ratings, path):
    ratings = ratings.rename(
        columns={
            "userId": "user_id:token",
            "itemId": "item_id:token",
            "Rating": "rating:float",
            "timestamp": "timestamp:float",
        }
    )
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    ratings.to_csv(path, sep="\t", index=False)
    return ratings


save_inter(new_ratings1, "./processed/movie/movie.inter")
save_inter(new_ratings2, "./processed/music/music.inter")
save_inter(new_ratings3, "./processed/book/book.inter")

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [ ]:
path = "./processed/douban/douban.inter"
data_df = pd.read

In [ ]:
# music domain 0 movie domain 1 all filter user number :110073, item number :44741
source_csv = "amazon14/reviews_Movies_and_TV_5.json.gz"
target_csv = "amazon14/reviews_Books_5.json.gz"

u1, i1, rating_num1, user_unique1, ratings1 = filter_data(source_csv)
u2, i2, rating_num2, user_unique2, ratings2 = filter_data(target_csv)
# save_csv_name = "music_movie_all.csv"
print(u1, i1, rating_num1)
print(u2, i2, rating_num2)
print(len(list(set(list(ratings1["userId"].unique())) & set(list(ratings2["userId"].unique())))))

In [ ]:
filter_ratings1, filter_ratings2 = filter_user(ratings1, ratings2)  # del overlap user < 5
filter_ratings1, filter_ratings2 = filter_item(filter_ratings1, filter_ratings2)  # del overlap user < 5

# filter_ratings1 = filter_ratings1.loc[filter_ratings1["Rating"] >= 3.0]
# filter_ratings2 = filter_ratings2.loc[filter_ratings2["Rating"] >= 3.0]

print(
    len(list(filter_ratings1["userId"].unique())), len(list(filter_ratings1["itemId"].unique())), len(filter_ratings1)
)
print(
    len(list(filter_ratings2["userId"].unique())), len(list(filter_ratings2["itemId"].unique())), len(filter_ratings2)
)
print(len(list(set(list(filter_ratings1["userId"].unique())) & set(list(filter_ratings2["userId"].unique())))))

In [ ]:
sample_ratings1, sample_ratings2 = sample_data(filter_ratings1, filter_ratings2, 1.0, 1.0)
print(len(sample_ratings1), len(sample_ratings2))

In [148]:
sample_ratings1 = sample_ratings1.rename(
    columns={
        "userId": "user_id:token",
        "itemId": "item_id:token",
        "Rating": "rating:float",
        "timestamp": "timestamp:float",
    }
)
sample_ratings2 = sample_ratings2.rename(
    columns={
        "userId": "user_id:token",
        "itemId": "item_id:token",
        "Rating": "rating:float",
        "timestamp": "timestamp:float",
    }
)


if not os.path.exists("./processed/movie"):
    os.makedirs("./processed/movie", exist_ok=True)
if not os.path.exists("./processed/book"):
    os.makedirs("./processed/book", exist_ok=True)
sample_ratings1.to_csv("./processed/movie/movie.inter", sep="\t", index=False)
sample_ratings2.to_csv("./processed/book/book.inter", sep="\t", index=False)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [3]:
ratings = pd.read_csv("./processed/netflix/netflix.inter", sep="\t")
#

In [5]:
ratings.columns = ["userId", "itemId", "Rating", "timestamp"]

In [ ]:
ratio = 0.001
samples = int(ratio * len(ratings["userId"].unique()))
sampled_user = random.sample(list(ratings["userId"].unique()), samples)
sampled_ratings = ratings[ratings["userId"].isin(sampled_user)]
sampled_ratings.shape
# samples1 = int(ratio1 * len(ratings1_nolap_user))
# samples2 = int(ratio2 * len(ratings2_nolap_user))

# sampled_user1 = random.sample(ratings1_nolap_user, samples1)
# sampled_user2 = random.sample(ratings2_nolap_user, samples2)

# new_ratings1 = ratings1[ratings1["userId"].isin(overlap_user + sampled_user1)]

In [ ]:
def reindex_ratings(ratings1, ratings2):
    user_unique1 = list(ratings1["userId"].unique())
    user_unique2 = list(ratings2["userId"].unique())
    item_unique1 = list(ratings1["itemId"].unique())
    item_unique2 = list(ratings2["itemId"].unique())
    user_dict = dict()
    item_dict = dict()
    for i in range(len(user_unique1)):
        user_dict[user_unique1[i]] = i
    for i in range(len(user_unique2)):
        user_dict[user_unique2[i]] = i + len(user_dict)
    for i in range(len(item_unique1)):
        item_dict[item_unique1[i]] = i
    for i in range(len(item_unique2)):
        item_dict[item_unique2[i]] = i + len(item_dict)
    ratings1["userId"] = ratings1["userId"].apply(lambda x: user_dict[x])
    ratings2["userId"] = ratings2["userId"].apply(lambda x: user_dict[x])
    ratings1["itemId"] = ratings1["itemId"].apply(lambda x: item_dict[x])
    ratings2["itemId"] = ratings2["itemId"].apply(lambda x: item_dict[x])
    print("all user number :{}, item number :{}".format(len(user_dict), len(item_dict)))
    return ratings1, ratings2


def find_dict(ratings):
    seq = defaultdict(list)
    uid = ratings["userId"].tolist()
    iid = ratings["itemId"].tolist()
    for i in range(len(uid)):
        seq[uid[i]].append(iid[i])
    return seq

In [ ]:
ratings1, ratings2 = reindex_ratings(ratings1, ratings2)
# print(ratings1,ratings2)

seq1 = find_dict(ratings1)
seq2 = find_dict(ratings2)
# print(seq1,seq2)
user_unique1 = list(ratings1["userId"].unique())
user_unique2 = list(ratings2["userId"].unique())
user_node, seq_d1, seq_d2, domain_id = [], [], [], []

for u_id_tmp in user_unique1:
    if len(seq1[u_id_tmp]) >= 5 and (len(seq2[u_id_tmp]) >= 5 or len(seq2[u_id_tmp]) == 0):
        user_node.append(u_id_tmp)
        seq_d1.append(seq1[u_id_tmp])
        seq_d2.append(seq2[u_id_tmp])
        domain_id.append(0)

for u_id_tmp in user_unique2:
    if len(seq2[u_id_tmp]) >= 5 and (len(seq1[u_id_tmp]) >= 5 or len(seq1[u_id_tmp]) == 0):
        user_node.append(u_id_tmp)
        seq_d1.append(seq1[u_id_tmp])
        seq_d2.append(seq2[u_id_tmp])
        domain_id.append(1)


dataframe = pd.DataFrame({"user_id": user_node, "seq_d1": seq_d1, "seq_d2": seq_d2, "domain_id": domain_id})
print(len(dataframe))
user_unique1 = list(dataframe["user_id"].unique())

save_csv_name = "music_movie_all.csv"

user_dict = dict()
for i in range(len(user_unique1)):
    user_dict[user_unique1[i]] = i
dataframe["user_id"] = dataframe["user_id"].apply(lambda x: user_dict[x])
print(dataframe)
dataframe.to_csv(save_csv_name, index=False, sep=",")

In [ ]:
df1 = pd.read_csv("dataset/ratings_Clothing_Shoes_and_Jewelry.csv", delimiter=",", encoding="latin1")
df2 = pd.read_csv("dataset/ratings_Sports_and_Outdoors.csv", delimiter=",", encoding="latin1")
df1.columns = ["userId", "itemId", "Rating", "timesteamp"]
df2.columns = ["userId", "itemId", "Rating", "timesteamp"]
len(list(set(list(df1["userId"].unique())) & set(list(df2["userId"].unique()))))

In [ ]:
from collections import OrderedDict

from prettytable import PrettyTable

data = OrderedDict(
    [
        ("0-hit@5", 0.0043),
        ("0-hit@10", 0.0072),
        ("0-hit@20", 0.0187),
        ("0-ndcg@5", 0.0024),
        ("0-ndcg@10", 0.0033),
        ("0-ndcg@20", 0.0062),
        ("0-mrr@5", 0.0018),
        ("0-mrr@10", 0.0022),
        ("0-mrr@20", 0.0029),
        ("0-precision@5", 0.0009),
        ("0-precision@10", 0.0007),
        ("0-precision@20", 0.0009),
        ("0-loss", 0.004844015357138097),
    ]
)

table = PrettyTable(["Metric", "Value"])
for key, value in data.items():
    table.add_row([key, value])

print(table)

In [12]:
import json
import os

import torch

data_save_path = "../save/SASRec.music/clusters_20.spc_6.len_50.dim_32/2024-09-20.18-46-07/checkpoints/last-ckpt"
model_save_path = "../checkpoint/SASRec-music.heads_1.layers_1.hiddens_32.inners_64.maxlen_50.batch_2048.epochs_100.pth"

In [13]:
config = json.load(open(os.path.join(data_save_path, "config.json")))

In [14]:
checkpoint = torch.load(model_save_path)
initial_state_dict = checkpoint["state_dict"]

In [15]:
config["item_emb"] = initial_state_dict["item_embedding.weight"].cpu().numpy().tolist()

In [16]:
json.dump(config, open(os.path.join(data_save_path, "config.json"), "w"), indent=4)
# torch.save(data_dict(detach=True), os.path.join(save_path, "data_dict"))

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [21]:
import os

import pandas as pd
from recbole.utils import init_seed

seed = 42
init_seed(seed, reproducibility=True)

data = "epinions"

df = pd.read_csv(f"./processed/{data}/{data}.inter", sep="\t")
df.head()

,user_id:token,item_id:token,rating:float,timestamp:float,price:float
0,tjtoolmanz,Samsung_226BW_Monitor,5.0,1195516800,299.0
1,Rufster,Samsung_226BW_Monitor,5.0,1181606400,300.0
2,jdproconsumer,Samsung_226BW_Monitor,4.0,1193443200,269.0
3,denverbrian,pr-Minolta_Dimage_X20_Digital_Camera,5.0,1099094400,120.0
4,endurance,pr-Minolta_Dimage_X20_Digital_Camera,4.0,1096761600,100.0


In [22]:
# Random Sample
def random_sample(fracs):
    for frac in fracs:
        sampled_groups = df["user_id:token"].drop_duplicates().sample(frac=frac, random_state=seed)
        sampled_df = df[df["user_id:token"].isin(sampled_groups)]
        # sampled_df = df.sample(frac=frac, random_state=seed)
        os.makedirs(f"processed/{data}_{frac}", exist_ok=True)
        sampled_df.to_csv(f"processed/{data}_{frac}/{data}_{frac}.inter", sep="\t", index=False)


# Longest Sample
def longest_sample(fracs):
    for frac in fracs:
        group_sizes = df.groupby("user_id:token").size().sort_values(ascending=False)
        num_groups_to_sample = int(len(group_sizes) * frac)
        top_groups = group_sizes.head(num_groups_to_sample)
        sampled_df = df[df["user_id:token"].isin(top_groups.index)]
        sampled_df.to_csv(f"processed/{data}_{frac}/{data}_{frac}.inter", sep="\t", index=False)

In [23]:
fracs = [0.005, 0.03, 0.1, 0.3, 0.5, 0.8, 1.0]
random_sample(fracs)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [67]:
rate_size_dic_u = df.groupby("user_id:token").size()
rate_size_dic_i = df.groupby("item_id:token").size()
rate_size_dic_u

user_id:token
1      272
2       62
3       54
4       24
5      175
      ... 
939     49
940    107
941     22
942     79
943    168
Length: 943, dtype: int64

In [ ]:
choosed_index_del_u = rate_size_dic_u[rate_size_dic_u < 5].index
choosed_index_del_i = rate_size_dic_i[rate_size_dic_i < 5].index
choosed_index_del_u

In [ ]:
core_df = df[~df["item_id:token"].isin(list(choosed_index_del_i))]
core_df = core_df[~df["user_id:token"].isin(list(choosed_index_del_u))]

len(df), len(core_df)

In [6]:
core_df.to_csv("./processed/book/book.inter", sep="\t", index=False)